# 5. Clock Pendulum with Air Drag and Joint Friction

This notebook builds on the previous one by introducing a nonlinear damping effect through Coulomb friction. Students will be able to work with both a linear and nonlinear version of the same system (a clock pendulum) in order to compare the free response in both cases.

After the completion of this assignment students will be able to:

- identify the function that governs the decay envelope
- compare this non-linear behavior to the linear behavior
- estimate the period of oscillation
- compute the free response of a non-linear system with viscous and coulomb damping

![](fig/05/coulomb-pendulum.png)

A common source of damping and energy dissipation is friction, which is specific type of damping. With viscous damping, we had a torque that was linearly proportional to the angular velocity:

$$T_f = c \dot{\theta} l^2 $$

This simple source of energy dissipation is a reasonable mathematical model for many phenomena, but it isn't often a good model for friction. One very useful model of friction is [Coulomb friction](https://en.wikipedia.org/wiki/Coulomb_damping). Coulomb friction behaves more like:

$$T_f = \begin{cases} -\mu F_N & \dot{\theta} > 0 \\ 0 & \dot{\theta} = 0 \\ \mu F_N & \dot{\theta} < 0 \end{cases}$$

where $\mu$ is the coefficient of sliding friction and $N$ is the normal force. Here the damping force is constant, always working against the motion of the system.

This can also be more simply written using the [signum function](https://en.wikipedia.org/wiki/Sign_function):

$$ T_f = -\mu F_n \text{sgn}\left( \dot{\theta} \right)$$

To start import some of the common packages we will need:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook

# Linear vs Nonlinear

`resonance` has a version of the clock pendulum with the viscous damping (a model of air drag) and Coulomb friction (a model of joint friction). The first is a [linear system](https://en.wikipedia.org/wiki/Linear_system) due to the torque being linear in the velocity and the second is a [nonlinear system](https://en.wikipedia.org/wiki/Nonlinear_system) because the torque being nonlinear with respect to the velocity.

In [ ]:
from resonance.nonlinear_systems import ClockPendulumSystem as NonLinPendulum
from resonance.linear_systems import ClockPendulumSystem as LinPendulum

In [ ]:
non_lin_sys = NonLinPendulum()

In [ ]:
lin_sys = LinPendulum()

In [ ]:
non_lin_sys.constants

In [ ]:
lin_sys.constants

**Exercise**

Simulate each system with an initial angle of 1.0 degrees and plot the trajectory of the angle from each response on a single plot to see if there are any differences.

In [ ]:
initial_angle = np.deg2rad(1.0)
non_lin_sys.coordinates['angle'] = initial_angle
lin_sys.coordinates['angle'] = initial_angle

duration = 5.0
non_lin_traj = non_lin_sys.free_response(duration)
lin_traj = lin_sys.free_response(duration)

In [ ]:
# write your answer here

In [ ]:
fig, ax = plt.subplots(1, 1)

ax.plot(non_lin_traj.index, non_lin_traj.angle)
ax.plot(lin_traj.index, lin_traj.angle)

ax.legend(['Nonlinear', 'Linear'])

**Exercise**

Create a function called `compare_lin_to_nonlin` that accepts a single angle value in degrees as the only argument and creates a plot just like the one you made above. Don't forget axis labels and a legend so that we can tell the two lines apart. It should look something like:

```python
def compare_lin_to_nonlin(initial_angle):
    # write your code here (hint copy most of it from above and modify)
```

In [ ]:
def compare_lin_to_nonlin(initial_angle):
    initial_angle = np.deg2rad(initial_angle)
    non_lin_sys.coordinates['angle'] = initial_angle
    lin_sys.coordinates['angle'] = initial_angle
    non_lin_traj = non_lin_sys.free_response(duration)
    lin_traj = lin_sys.free_response(duration)
    fig, ax = plt.subplots(1, 1)
    ax.plot(non_lin_traj.index, np.rad2deg(non_lin_traj.angle))
    ax.plot(lin_traj.index, np.rad2deg(lin_traj.angle))
    ax.legend(['Nonlinear', 'Linear'])
    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Angle [deg]')

In [ ]:
# write your answer here

**Exercise**

Try out some angles from 0 to 180 degrees and view the graphs to see if there is anything interesting.

In [ ]:
compare_lin_to_nonlin(1)
compare_lin_to_nonlin(25)
compare_lin_to_nonlin(87)
compare_lin_to_nonlin(130)

In [ ]:
# write your answer here

**Exercise**

Make a plot of initial angle versus period for the nonlinear pendulum. Report on your what you learn from this plot. The code should look something like:

```python
periods = []
angles = np.deg2rad(np.linspace(0, 90))
for angle in angles:
    # fill in the loop here
    
fig, ax = plt.subplots(1, 1)
ax.plot(np.rad2deg(angles), periods)
ax.set_ylabel('Period [s]')
ax.set_xlabel('Initial Angle [deg]')
```

In [ ]:
from resonance.functions import estimate_period
periods = []
angles = np.deg2rad(np.linspace(0, 90))
for angle in angles:
    non_lin_sys.coordinates['angle'] = angle
    traj = non_lin_sys.free_response(5.0)
    periods.append(estimate_period(traj.index, traj.angle))
    
fig, ax = plt.subplots(1, 1)
ax.plot(np.rad2deg(angles), periods)
ax.set_ylabel('Period [s]')
ax.set_xlabel('Initial Angle [deg]')

In [ ]:
# write your answer here

# Adding Damping

Below we add a small amount of viscous damping to the linear pendulum and a small amount of Coulomb friction to the nonlinear pendulum. If you compare the trajectories of the angle at an initial angle of 5 degrees you see some differences.

In [ ]:
lin_sys.constants['viscous_damping'] = 0.1  # Ns/m
non_lin_sys.constants['coeff_of_friction'] = 0.002  # Nm

In [ ]:
compare_lin_to_nonlin(5.0)

# Curve Fitting an Exponential Decay Function

In [ ]:
non_lin_traj = non_lin_sys.free_response(5.0, sample_rate=500)

In [ ]:
from scipy.optimize import curve_fit

**Exercise**

Use your curve fitting function for $\theta(t) = A e^{\lambda t} \cos(\omega t + \phi)$ and see how well it fits the nonlinear trajectory.

```python
def exp_decayed_oscillation(times, amplitude, decay_constant, frequency, phase_shift):
    # write your function here and return the answer
```

In [ ]:
def exp_decayed_oscillation(times, amplitude, decay_constant, frequency, phase_shift):
    return amplitude * np.exp(decay_constant * times) * np.cos(frequency * times + phase_shift)

In [ ]:
# write your answer here

Since there is something funny going on at the end of the simulation, only fit to the first 3 seconds of data.

In [ ]:
popt, pcov = curve_fit(exp_decayed_oscillation,
                       non_lin_traj[:3.0].index, non_lin_traj[:3.0].angle,
                       p0=(5.0, -0.001, 2 * np.pi, 0.0))

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(non_lin_traj.index, non_lin_traj.angle)
best_fit_angle = exp_decayed_oscillation(non_lin_traj.index, popt[0], popt[1], popt[2], popt[3])
ax.plot(non_lin_traj.index, best_fit_angle)

**Exercise**

Now try a function that looks like:

$$ \theta(t) = (m t + A) \cos(\omega t + \phi) $$

This is a linear decaying function instead of an exponentially decaying function.

```python
def lin_decayed_oscillation(times, m, a, frequency, phi):
    # write function here and return result
```

In [ ]:
def lin_decayed_oscillation(times, m, a, frequency, phi):
    return (m * times + a) * np.cos(frequency * times + phi)

In [ ]:
# write your answer here

In [ ]:
popt, pcov = curve_fit(lin_decayed_oscillation,
                       non_lin_traj[:3.0].index, non_lin_traj[:3.0].angle,
                       p0=(-0.1, 0.07, 2 * np.pi, 0.0))

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(non_lin_traj.index, non_lin_traj.angle, '.')
best_fit_angle = lin_decayed_oscillation(non_lin_traj.index, popt[0], popt[1], popt[2], popt[3])
ax.plot(non_lin_traj.index, best_fit_angle)

# Visualization

Finally, visualize the animation with different friction coefficients to see the behavior.

In [ ]:
non_lin_traj = non_lin_sys.free_response(5.0, sample_rate=100)

In [ ]:
non_lin_sys.animate_configuration(interval=1000/100)  # interval should be in milliseconds